<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Irene_Nsengumukiza_Hackathon_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation zone**

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

In [ ]:
!pip install -U easynmt

**All required imports**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from fastapi import FastAPI, Response
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from starlette.responses import HTMLResponse
import json
from easynmt import EasyNMT

**other prequisties**

In [ ]:
!ngrok authtoken 2HJPvK5e4VxcduBjO5XSc0LqqCR_5zV3b5gBcKuNMUnxdpfuP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#PART 1: WEB SCRAPING OF JOB IN RWANDA

In [ ]:
# SOUP
url = 'https://www.jobinrwanda.com/jobs/featured'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [ ]:
# LINKS OF ALL FEATURED CONTENT ON WEB
job_titles = {}

parent_divs = soup.find_all('div', class_ = 'card-body p-2')
featured_links = []

for index,div in enumerate(parent_divs):
  anchor = div.find('a')
  output_link = 'https://www.jobinrwanda.com'+anchor['href']
  job_titles[index] = anchor.find('h5', class_ = 'card-title').get_text()
  featured_links.append(output_link)
job_titles = list(job_titles.values())

In [ ]:
# EACH LINK SOUP
soups2 = {}
for index,link in enumerate(featured_links):
  SOUP = BeautifulSoup(requests.get(link).content, 'html.parser')
  soups2[index] = SOUP

In [ ]:
# DIVS FOR EACH FEATURE CONTENT
description_divs = {}
job_summary = {}
for index,level2_soup in enumerate(list(soups2.values())):
  description_divs[index] = level2_soup.find('div', class_ = 'clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item')
  ul = level2_soup.find('ul', class_ = 'list-group list-group-flush')
  info = {}
  for li in ul.children:
    try:
      summary = re.sub('\s{2,}|\n',' ',li.text) # both key and value together as str
      # separate key & value to get key:value dictionary
      if not 'apply' in summary.lower():
        if 'viewed' in summary:
          times = re.search('\d+', summary).group()
          info['views'] = times
        else:
          k,value = re.split(':', summary)[:2]
          info[k.lstrip()] = value
    except AttributeError:
      pass
  # info["application link"] = app_link
  # # print(info)
  job_summary[index] = info


In [ ]:
# FULL CONTENT OF EACH FEATURE
full_description = {}
for index,div in enumerate(list(description_divs.values())):
  single_div_content = []
  for para in div.find_all('p', class_ = 'text-align-justify'):
    single_div_content.append(para.get_text())
  full_description[index] = re.sub('\n|\\xa0','','\n'.join(single_div_content))

In [ ]:
# CREATE THE JOB ROW TO BE USED IN THE DATAFRAME
job_info_rows = []
for index in list(full_description.keys()):
  summary_info = list(job_summary[index].values())
  descr_text = [full_description[index]]
  job_info_rows.append(summary_info + descr_text)


In [ ]:
# CREATE DATAFRAME
columns = list(job_summary[0].keys()) + ['Description']
df1 = pd.DataFrame(job_info_rows, columns = columns)
df1['job title'] = job_titles
df1['job title'] = df1['job title'].str.replace('\n','',regex=True).astype('str')

In [ ]:
df1.head()

In [ ]:
# FILTER FOR ONLY IT RELATED JOBS & CONSULTANCY
IT_based_jobs = df1.copy()
IT_based_jobs['is IT job'] = False

for index in range(len(IT_based_jobs)):
  if re.findall('Computer|IT|Software', IT_based_jobs.iloc[index]['Sector']):
    IT_based_jobs['is IT job'][index] = True

IT_based_jobs = IT_based_jobs[IT_based_jobs['is IT job'] == True]

In [ ]:
# display IT jobs 
IT_based_jobs.head()

#PART 2 : WEBSCRAP umucyo.gov.rw

In [ ]:
url = 'https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do'

text = """"""
with open('/content/drive/MyDrive/NLP fellowship/Umucyo.txt', 'r') as f:
  data = f.readlines()
  for line in data:
    text += line

# umucyo_soup = BeautifulSoup(requests.get(url).content, 'html.parser')
umucyo_soup = BeautifulSoup(text, 'html.parser')
table = umucyo_soup.find('table', class_ = 'article_table mb10')

# CREATE COLUMNS FOR THE DATAFRAME
head_row = table.find('thead').find('tr')
titles = []
for th in head_row.find_all('th'):
  titles.append(re.sub('\r|\n|\t','',th.text))
titles = titles[1:]

In [ ]:
# MAKE DATAFRAME BODY
body_rows = table.find('tbody').find_all('tr') 
full_data = []
for row in body_rows:
  ls = []
  for detail in row.find_all('td'):
    ls.append(re.sub('\n|\r|\t|\s', '',detail.text))
  full_data.append(ls[1:])


In [ ]:
# CREATE UMUCYO DATAFRAME
umucyo_df = pd.DataFrame(full_data, columns = titles)

In [ ]:
umucyo_df

#PART 3 : DISPLAY DATA ON THE BROWSER

In [ ]:
model = EasyNMT('opus-mt')
app = FastAPI(title = "to get something you never had you have do something you never did")

**PART 3.1. DISPLAY JSON DATA OF DATAFRAMES**

In [ ]:
@app.get('/')
def homepage():
  content = f"""<html>
                    <h1>Welcome to Irene Nsengumukiza API for the first hackathon</h1>
                    <p>
                      It does not matter how slowly you go as long as you do not stop. <br> 
                      Never give up, for that is just the place and time that the tide will turn. <br>
                      You just can't beat the person who won't give up. <br> 
                      There is no failure except in no longer trying.
                    </p>
                    <p>
                      <strong>never give up! \U0001F680 \U0001F680 \U0001F680 </strong>
                    </p>
                </html>
              """ 
  return HTMLResponse(content)

In [ ]:
# DISPLAY ALL IT JOBS ON THE BROWSER
@app.get('/IT')
def show(orient:str='index'):
    return Response(IT_based_jobs.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# SHOW TO THE BROWSER ALL INFORMATIONS RETRIEVED FROM JOBINRWANDA
@app.get('/jobs')
def showall(orient:str='index'):
    return Response(df1.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# SHOW ALL UMUCYO INFORMATION TO THE BROWSER
@app.get('/umucyo_tenders')
def show_umucyo(orient:str='index'):
    return Response(umucyo_df.to_json(orient = orient), media_type = 'application/json')

**PART 3.2. TRANSLATE ALL IT RELATED JOBS DATAFRAME COLUMNS DESCRIPTION AND JOB TITLE**

In [ ]:
# TRANSLATE ALL IT RELATED JOBS IN DIFFERENT LANGUANGES
prev_lang = 'en'
@app.get('/translate')
def dr(lang:str=''):
  global prev_lang
  success = True
  if lang:
    try:
      d = []
      f = []
      for index in range(len(IT_based_jobs)):
        text1 = IT_based_jobs.iloc[index]['Description']
        text2 = IT_based_jobs.iloc[index]['job title']
        d.append(model.translate(text1, source_lang = prev_lang, target_lang = lang))
        f.append(model.translate(text2, source_lang = prev_lang, target_lang = lang))

        if index != len(IT_based_jobs) - 1:
          print(f"{index} / {len(IT_based_jobs)}")
           
      IT_based_jobs['job title'] = f
      IT_based_jobs['Description'] = d
      success = True
      prev_lang = lang
    except:
      error_message = f"""<html>
                            <h2>
                              Oops! Something went wrong. <br> Try to choose other language maybe!.
                            </h2>
                          </html>"""
      success = False
      return HTMLResponse(error_message)
  if success:
    return Response(IT_based_jobs.to_json(orient = 'index'), media_type = 'application/json')

In [ ]:
# RUN THE SERVER
pipe = ngrok.connect(8000)
print("PUBLIC URL: ", pipe.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

In [ ]:
cs = IT_based_jobs.to_csv( "mycsv.csv")